The Lake Model as preppared by the EMA Workbench:
Kwakkel, Jan H. "The Exploratory Modeling Workbench: An open source toolkit for exploratory modeling, scenario discovery, and (multi-objective) robust decision making." Environmental Modelling & Software 96 (2017): 239-250.

Peterson, G.D., Carpenter, S.R. and Brock, W.A. (2003), UNCERTAINTY AND THE MANAGEMENT OF MULTISTATE ECOSYSTEMS: AN APPARENTLY RATIONAL ROUTE TO COLLAPSE. Ecology, 84: 1403-1411. https://doi.org/10.1890/0012-9658(2003)084[1403:UATMOM]2.0.CO;2

In [48]:

import random as rand
from scipy.optimize import brentq as root

import numpy as np

In [47]:



# Construct the lake problem
def lake_problem(pollution_limit_low,
                 pollution_limit_high,
         b = 0.42,        # decay rate for P in lake (0.42 = irreversible)
         q = 2.0,         # recycling exponent
         mean = 0.02,     # mean of natural inflows
         stdev = 0.001,   # standard deviation of natural inflows
         alpha = 0.4,     # utility from pollution
         delta = 0.98,    # future utility discount rate
         myears = 150,
         nsamples = 100): # monte carlo sampling of natural inflows
    pollution_limit = [(rand.random()*pollution_limit_high)+pollution_limit_low  for i in  range(myears)]
    Pcrit = root(lambda x: x**q/(1+x**q) - b*x, 0.01, 1.5)
    nvars = len(pollution_limit)
    X = np.zeros((nvars,))
    average_daily_P = np.zeros((nvars,))
    print(pollution_limit)
    decisions = np.array(pollution_limit)
    reliability = 0.0

    for _ in range(nsamples):
        X[0] = 0.0

        natural_inflows = np.random.lognormal(
                math.log(mean**2 / math.sqrt(stdev**2 + mean**2)),
                math.sqrt(math.log(1.0 + stdev**2 / mean**2)),
                size = nvars)

        for t in range(1,nvars):
            X[t] = (1-b)*X[t-1] + X[t-1]**q/(1+X[t-1]**q) + decisions[t-1] + natural_inflows[t-1]
            average_daily_P[t] += X[t]/float(nsamples)

        reliability += np.sum(X < Pcrit)/float(nsamples*nvars)

    max_P = np.max(average_daily_P)
    utility = np.sum(alpha*decisions*np.power(delta,np.arange(nvars)))
    inertia = np.sum(np.diff(decisions) > -0.02)/float(nvars-1)

    return (max_P, utility, inertia, reliability)

In [27]:
b : 'CWLFloatInput' = 0.33
q : 'CWLFloatInput' = 2
mean : 'CWLFloatInput' = 0.05
stdev : 'CWLFloatInput' = 0.004
delta : 'CWLFloatInput' = 0.99
alpha : 'CWLFloatInput' = 0.41
nsamples : 'CWLIntInput' = 150
myears : 'CWLIntInput' = 100
pollution_limit_low : 'CWLFloatInput'
pollution_limit_high : 'CWLFloatInput'

output_file : 'CWLFilePathOutput' = "./y.csv"

In [28]:
y = lake_problem(pollution_limit_low=pollution_limit_low,
    pollution_limit_high=pollution_limit_high,
    b= b,
    q=q,
    mean=mean,
    stdev=stdev,
    delta=delta,
    alpha=alpha,
    myears=myears,
    nsamples=nsamples,

   )

0.5


In [29]:
np.savetxt(output_file,y)

(4.558926902468055,
 10.205334373075656,
 0.9092666666666664,
 0.009999999999999966)

0.23282161272707502

In [15]:
c1, c2

(1.99, -1.5)